# Scraping delle pagine di una libreria online

Estrai da questa libreria online (https://books.toscrape.com/)  le seguenti informazioni dei libri:

- titolo
- valutazione in stelle
- prezzo
- disponibilità

Organizza infine i dati in formato strutturato e salvali nel file books.csv.


Ricorda di gestire le eccezioni per assicurarti che il tuo script possa evitare eventuali errori durante la richiesta o il parsing.

## Installazione di Selenium

Si riportano i passaggi necessari ad installare correttamente Selenium.

In [ ]:
from selenium import webdriver
from selenium.webdriver import ChromeService, Chrome

service = ChromeService(r'chromedriver-win64\chromedriver.exe')
driver = Chrome(service=service)

## Passaggi preliminari

Importazione delle librerie

In [13]:
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from typing import List, Dict, Any
import logging

Configurazione del logger

In [14]:
# Configurazione del logger
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Creazione di un handler per il file
file_handler = logging.FileHandler('app.log')
file_handler.setLevel(logging.DEBUG)

# Creazione di un formatter e aggiunta all'handler del file
file_formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
file_handler.setFormatter(file_formatter)

# Aggiungi l'handler del file al logger
logger.addHandler(file_handler)

Definizione della funzione necessaria ad estrarre le informazioni dei libri, dalla pagina corrente

In [15]:
def extract_books_from_page(driver: WebDriver) -> List[Dict[str, Any]]:
    """
    Estrae le informazioni sui libri da una pagina web.

    Args:
        driver (WebDriver): Il driver di Selenium utilizzato per navigare la pagina web.

    Returns:
        List[Dict[str, Any]]: Una lista di dizionari, ciascuno contenente le informazioni di un libro.
            Ogni dizionario ha le seguenti chiavi:
                - 'title': Il titolo del libro (str).
                - 'rating': La valutazione del libro (str).
                - 'price': Il prezzo del libro (str).
                - 'stock availability': La disponibilità in magazzino (str).
    """
    books = []
    book_elements = driver.find_elements(By.CLASS_NAME, 'product_pod')  # i libri si trovano in un tag article, con class product_pod
    for book in book_elements:
      try:
        #estrazione del titolo
        title_element = book.find_element(By.CSS_SELECTOR, 'h3 > a') if book.find_elements(By.CSS_SELECTOR, 'h3 > a') else 'No Title'
        title = title_element.get_attribute('title')
        #estrazione del prezzo
        price = book.find_element(By.CLASS_NAME, 'price_color').text if book.find_element(By.CLASS_NAME, 'price_color') else 'No Price'
        #estrazione dell'informazione sulla disponibilità in magazzino
        stock_availability = book.find_element(By.CSS_SELECTOR, 'p.instock.availability').text if book.find_element(By.CSS_SELECTOR, 'p.instock.availability') else 'Unknown'
        #estrazione del rating
        rating_element = book.find_element(By.CSS_SELECTOR, 'p.star-rating') if book.find_elements(By.CSS_SELECTOR, 'p.star-rating') else None
        if rating_element:
          rating_class = rating_element.get_attribute('class')
          #essendo la classe chiamata "star-rating Value", la valutazione è l'ultima parte del nome della classe
          rating = rating_class.split()[-1]
        else:
          rating = "No rating"

        # Verifica della coerenza e validità dei dati
        if title == 'No Title' or price == 'No Price':
            logger.warning(f"Libro con informazioni mancanti: title={title}, price={price}")

        books.append({'title': title ,'rating': rating, 'price': price, "stock availability":stock_availability})

      except Exception as e:
        logger.error(f"Errore durante l'estrazione dei dati di un libro: {e}")
    return books

Definizione di una funzione necessaria a navigare nelle altre pagine del sito, andando a cliccare sul tasto "prossima pagina" (in questo caso "next").

In [16]:
def get_next_page(driver: WebDriver) -> bool:
    """
  Naviga alla pagina successiva se il pulsante 'Next' è presente.

  Args:
      driver (WebDriver): Il driver di Selenium utilizzato per navigare la pagina web.

  Returns:
      bool: True se la navigazione alla pagina successiva è riuscita, False altrimenti.
  """
    try:
        next_button = driver.find_element(By.CSS_SELECTOR, 'li.next > a')  # l'elemento desiderato si trova dentro un <a> che è figlio diretto di un elemento <li> con classe next
        if next_button:
            next_button.click()
            return True
    except Exception as e:
        logger.error(f"Error navigating to next page: {e}")
    return False

## Esecuzione dello script di estrazione

Si fa la richiesta all'url del sito. Per ogni pagina si estraggono le informazioni dei libri presenti. Estratte tutte le informazioni necessarie si passa alla pagina successiva, al fine di raccogliere informazioni su tutto il catalogo di libri.

In [17]:
url = "https://books.toscrape.com/"

In [18]:
#si definisce una lista di appoggio in cui inserire tutte le informazioni estratte da tutti i libri del catalogo
all_books = []
#si fa la richiesta della pagina
driver.get(url)

while True:
  try:
    #si aspetta che vengano visualizzati gli elementi con classe product_pod, che contengono le informazioni dei libri
    WebDriverWait(driver, 10).until(
    EC.visibility_of_all_elements_located((By.CLASS_NAME, "product_pod"))
)
    #si estraggono le informazioni dei libri della pagina
    books_info = extract_books_from_page(driver)
    #si aggiungono all lista d'appoggio
    all_books.extend(books_info)

    #si cerca di andare alla pagina successiva
    if not get_next_page(driver):
        break

    #si inserisce un ritardo per evitare di sovraccaricare il server
    time.sleep(1)

  except Exception as e:
    logger.error(f"Error during scraping: {e}")
    break

driver.quit()

## Salvataggio dei dati in formato csv

Si organizzano i dati estratti in formato strutturato grazie ad un dataframe pandas e si salvano nel file books.csv

In [19]:
df = pd.DataFrame(all_books)
df.to_csv("books.csv", index= False) #si salvano i dati nel file csv richiesto, escludendo la colonna dell'indice